In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import pandas as pd
import scipy.sparse as sp
import numpy as np
from collections import defaultdict
from typing import Dict, List
import copy
import os

ModuleNotFoundError: No module named 'scipy'

In [3]:
user_min = 5
item_min = 5

df = pd.read_csv("../inputs/ml-100k/u.data",
                 header=None,
                 sep="\t",
                 names=["user_id", "item_id", "rating", "time"])
df.head()

,user_id,item_id,rating,time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
print('First pass')
print('num_users = {}'.format(df["user_id"].unique().size))
print('num_items = {}'.format(df["item_id"].unique().size))
print('df_shape  = {}'.format(df.shape))

First pass
num_users = 943
num_items = 1682
df_shape  = (100000, 4)


In [5]:
user_counts = df["user_id"].value_counts()
user_counts.head()

405    737
655    685
13     636
450    540
276    518
Name: user_id, dtype: int64

In [6]:
item_counts = df["item_id"].value_counts()
item_counts.head()

50     583
258    509
100    508
181    507
294    485
Name: item_id, dtype: int64

In [7]:
df = df[df["user_id"] >= user_min]
df = df[df["item_id"] >= item_min]

In [8]:
print('Second pass')
print('num_users = {}'.format(df["user_id"].unique().size))
print('num_items = {}'.format(df["item_id"].unique().size))
print('df_shape  = {}'.format(df.shape))

Second pass
num_users = 939
num_items = 1678
df_shape  = (98711, 4)


In [9]:
print("Normalizing temporal values...")
mean_time = df["time"].mean()
std_time = df["time"].std()
df["time"] = (df["time"] - mean_time) / std_time

Normalizing temporal values...


In [10]:
print("Constructing datasets ...")
training_set: Dict[int, List] = defaultdict(list)

Constructing datasets ...


In [11]:
num_users = 1
num_items = 1
item_to_idx: Dict[int, int] = {}
user_to_idx: Dict[int, int] = {}
idx_to_item: Dict[int, int] = {}
idx_to_user: Dict[int, int] = {}

In [12]:
for row in df.itertuples():
    # New item
    if row.item_id not in item_to_idx:
        item_to_idx[row.item_id] = num_items
        idx_to_item[num_items] = row.item_id
        num_items += 1

    # New user
    if row.user_id not in user_to_idx:
        user_to_idx[row.user_id] = num_users
        idx_to_user[num_users] = row.user_id
        num_users += 1

    # Converts all ratings to positive implicit feedback
    training_set[user_to_idx[row.user_id]].append(
        (item_to_idx[row.item_id], row.time))

for user in training_set:
    training_set[user].sort(key=lambda x: x[1])

In [16]:
training_set[1]

[(1, -0.42953650834330054),
 (282, -0.42953650834330054),
 (485, -0.42953650834330054),
 (373, -0.42952301844949925),
 (745, -0.42951852181823214),
 (460, -0.4295001605738915),
 (732, -0.4294756164615586),
 (11, -0.4294188464918115),
 (665, -0.4294143498605444),
 (1037, -0.42940666811546313),
 (643, -0.42939542653729534),
 (370, -0.429390555186756),
 (176, -0.429390555186756),
 (382, -0.4293858711958528),
 (658, -0.4293858711958528),
 (506, -0.4293858711958528),
 (425, -0.42937837681040764),
 (82, -0.42937837681040764),
 (322, -0.42937331810023216),
 (218, -0.42937331810023216),
 (314, -0.42937331810023216),
 (417, -0.4293691961882373),
 (91, -0.4293690088286012),
 (520, -0.42936526163587857),
 (481, -0.4293613270835199),
 (542, -0.4293562683733444),
 (284, -0.4293562683733444),
 (295, -0.42934821190899086),
 (85, -0.4293480245493547),
 (426, -0.4293480245493547),
 (358, -0.4293480245493547),
 (826, -0.4293364082519147),
 (432, -0.4293364082519147),
 (626, -0.4293291012261057),
 (650, 